In [1]:
import os
import shutil
from glob import glob
from osgeo import gdal
from statistics import mode
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [2]:
if not os.path.exists(os.getcwd()+"\\mosaic_images"):
    stack_folder = os.makedirs(os.getcwd()+"\\mosaic_images")

In [3]:
stacked_images = os.getcwd() + "\\stacked_images"
month_list = glob(stacked_images+"\\*_archive")
month_list

['C:\\Users\\Hello\\Documents\\remote-sensing\\APIs\\SentinelSat\\stacked_images\\Godavari2016_archive',
 'C:\\Users\\Hello\\Documents\\remote-sensing\\APIs\\SentinelSat\\stacked_images\\Godavari2017_archive',
 'C:\\Users\\Hello\\Documents\\remote-sensing\\APIs\\SentinelSat\\stacked_images\\Godavari2018_archive',
 'C:\\Users\\Hello\\Documents\\remote-sensing\\APIs\\SentinelSat\\stacked_images\\Krishna2016_archive',
 'C:\\Users\\Hello\\Documents\\remote-sensing\\APIs\\SentinelSat\\stacked_images\\Krishna2017_archive',
 'C:\\Users\\Hello\\Documents\\remote-sensing\\APIs\\SentinelSat\\stacked_images\\Krishna2018_archive']

In [4]:
#curating files w.r.t date
paths = {}
for month in month_list:
    stacks = glob(month+"\\*.tif")
    for stack in stacks:
        date = month.split('\\')[-1].split('_')[0][:-4]+(stack.split("\\")[-1].split('.')[0].split('_')[-1][:8])
        value = []
        if date in paths.keys():
            value = paths.get(date)
        value.append(stack)
        paths.update({date: value})

In [ ]:
#filter by tiles count
maxTiles = int(input("How many tiles to be merged? "))
pop = []
for key, value in paths.items():
    if len(value) < maxTiles:
        pop.append(key)
for key in pop:
    paths.pop(key)

In [ ]:
#delete improper scenes
for key, value in paths.items():
    if len(value) > maxTiles:
        print("There are extra tiles. Creating the best match " + str(key) + "... Recheck the mosaic for this")
        black_counts = []
        for path in value:
            ds = gdal.Open(path)
            nir = np.array(ds.GetRasterBand(1).ReadAsArray())
            blue = np.array(ds.GetRasterBand(2).ReadAsArray())
            green = np.array(ds.GetRasterBand(3).ReadAsArray())
            red = np.array(ds.GetRasterBand(4).ReadAsArray())
            stacked_array = np.dstack((nir, blue, green, red))
            print(path.split('\\')[-1].split('.')[0])
            black_count = 0
            black = [0,0,0,0]
            for row in stacked_array:
                black_count = black_count + row.tolist().count(black)    
            black_counts.append(black_count)
        zipped = zip(value, black_counts)
        res = list(sorted(zipped, key = lambda x: x[1]))
        vals = []
        for file in res:
            vals.append(file[0])
        paths[key] = vals[:maxTiles]

In [6]:
#Mosaic

tot = len(paths.keys())
i = 1
for key, value in paths.items():
    dst = os.getcwd() + "\\mosaic_images\\" + str(key) + ".tif"
    srcs = " ".join(value)
    print("Mosaicing " + dst)
    os.system("gdal_merge.py -v " + srcs + " -o " + dst)
    if os.path.exists(dst):
        print("(" + str(i) + "/" + str(tot) + ") " + str(key)+" - saved to mosaic_images...")  
    i = i + 1

(1/3) Godavari20180225 - saved to mosaic_images...
(2/3) Krishna20170210 - saved to mosaic_images...
Mosaicing C:\Users\Hello\Documents\remote-sensing\APIs\SentinelSat\mosaic_images\Krishna20180302.tif
(3/3) Krishna20180302 - saved to mosaic_images...


In [ ]:
#show, show all images in folder

In [ ]:
#cleanup